# Tweet text Preprocessing

In [1]:
import pandas as pd
import json
import os
import matplotlib.pyplot as plt

import spacy
from spacy.lang.id import Indonesian

nlp = Indonesian()  # use directly
stopwords = spacy.lang.id.stop_words.STOP_WORDS
stopwords.add("nya")
stopwords.add("ya")
stopwords.add("cc")
stopwords.add("sih")
stopwords.add("an")
stopwords.add("dr")
stopwords.add("ri")
stopwords.add("lho")
stopwords.add("loh")

pd.set_option('display.max_colwidth', -1)

In [2]:
data = pd.read_csv(os.path.join(os.pardir,os.pardir,"data","04_out","50-KCore-tweet.csv"))

In [3]:
df = data

## Regular Expression

In [4]:
df['clean'] = df['text'].str.lower()
# df['clean'] = df['clean'].str.normalize('NFKD')
df['clean'] = df['clean'].str.replace(r"htt\S+|pic\.\S+|pict\S+|www\S+|t\.\S+|bit\.\S+|goo\.\S+", " ") # remove http, https, pic, 
df['clean'] = df['clean'].str.replace(r"rt @[\w_]+", " ") # remove rt @username
df['clean'] = df['clean'].str.replace(r"[@#][a-z0-9_]+", " ") # remove mention and hashtag
df['clean'] = df['clean'].str.replace(r"[^a-z]"," ") # remove non alphabetic
df['clean'] = df['clean'].str.replace(r"tidak\s+", "tidak_")
df['clean'] = df['clean'].str.replace(r"(.)\1{2,}", r"\1") # replace 3 repeated character in words into 1 character
df['clean'] = df['clean'].str.replace(r"(^| ).(( ).)*( |$)"," ") # remove 1 standalone character
df['clean'] = df['clean'].str.replace(r"\s\s+"," ") # remove multi space after regex
# https://dspace.uii.ac.id/bitstream/handle/123456789/12430/Tugas%20Akhir.pdf?sequence=1&isAllowed=y
df['clean'] = df['clean'].str.lstrip()

In [5]:
df['clean'].tail()

14543    yang namanya sumpah dan janji sdh tak berharga lagi di rezim ini moral penyelenggaraan pemilu perlu dipertanyakan                                                                                                                                                    
14544    mana ada wasit terserah pemain wasit itu memimpin pertandingan pembuat keputusan halo angkat tangan dari sekarang bila tidak_mampu jadi wasit masih banyak wni di negeri ini yang lebih mampu dan kompeten menjadi wasit dalam pemilu kali ini                       
14545    berarti bukan kampanye lihat di uu ttg pemilu bahwa yg namanya kampanye itu menyampaikan visi misi atau saya yang salah baca ya                                                                                                                                      
14546    diskusi akhir tahun kemarin dan sempat bahas masalah kotak kardus suara punya ada hal menarik bhw kotak suara kardus bukan masalah bahkan terbuat dari daun pisangpun bukan masala

## Tokenization

In [6]:
def df_tokenizer(tweets_df):
    tokenized = [token.text for token in nlp(tweets_df)]
    return tokenized

In [7]:
tokenized = df['clean'].apply(df_tokenizer)

In [8]:
tokenized.tail()

14543    [yang, namanya, sumpah, dan, janji, sdh, tak, berharga, lagi, di, rezim, ini, moral, penyelenggaraan, pemilu, perlu, dipertanyakan]                                                                                                                                                                        
14544    [mana, ada, wasit, terserah, pemain, wasit, itu, memimpin, pertandingan, pembuat, keputusan, halo, angkat, tangan, dari, sekarang, bila, tidak_mampu, jadi, wasit, masih, banyak, wni, di, negeri, ini, yang, lebih, mampu, dan, kompeten, menjadi, wasit, dalam, pemilu, kali, ini]                       
14545    [berarti, bukan, kampanye, lihat, di, uu, ttg, pemilu, bahwa, yg, namanya, kampanye, itu, menyampaikan, visi, misi, atau, saya, yang, salah, baca, ya]                                                                                                                                                     
14546    [diskusi, akhir, tahun, kemarin, dan, sempat, bahas, masalah, ko

## Slang Word Normalization

In [9]:
def slang(tokenized_):
    slang_word = json.loads(open('slang.json','r').read())
    for index in range(len(tokenized_)):
        for key, value in slang_word.items():
            for v in value:
                if tokenized_[index] == v:
                    tokenized_[index] = key
                else:
                    continue
    value = " ".join(str(v) for v in tokenized_) #join as string
    return value

In [10]:
slang_cleaned = tokenized.apply(slang)

In [11]:
slang_cleaned.tail()

14543    yang nama sumpah dan janji sudah tidak berharga lagi di rezim ini moral penyelenggaraan pemilu perlu dipertanyakan                                                                                                                                                    
14544    mana ada wasit terserah pemain wasit itu memimpin pertandingan pembuat keputusan halo angkat tangan dari sekarang bila tidak_mampu jadi wasit masih banyak wni di negeri ini yang lebih mampu dan kompeten menjadi wasit dalam pemilu kali ini                        
14545    berarti bukan kampanye lihat di undang-undang tentang pemilu bahwa yang nama kampanye itu menyampaikan visi misi atau saya yang salah baca ya                                                                                                                         
14546    diskusi akhir tahun kemarin dan sempat bahas masalah kotak kardus suara punya ada hal menarik bahwa kotak suara kardus bukan masalah bahkan terbuat dari daun pisangpun bukan m

## Lemmatization

In [12]:
def lemmatize(slang_cleaned_):
    lemmatized = [token.lemma_ for token in nlp(slang_cleaned_)]
    return lemmatized

In [13]:
lemmatized = slang_cleaned.apply(lemmatize)

In [14]:
lemmatized.tail()

14543    [yang, nama, sumpah, dan, janji, sudah, tidak, harga, lagi, di, rezim, ini, moral, selenggara, pemilu, perlu, dipertanyakan]                                                                                                                                                                         
14544    [mana, ada, wasit, serah, main, wasit, itu, pimpin, tanding, buat, putus, halo, angkat, tangan, dari, sekarang, bila, tidak_mampu, jadi, wasit, masih, banyak, wni, di, negeri, ini, yang, lebih, mampu, dan, kompeten, jadi, wasit, dalam, pemilu, kali, ini]                                       
14545    [arti, bukan, kampanye, lihat, di, undang, tentang, pemilu, bahwa, yang, nama, kampanye, itu, sampai, visi, misi, atau, saya, yang, salah, baca, ya]                                                                                                                                                 
14546    [diskusi, akhir, tahun, kemarin, dan, sempat, bahas, masalah, kotak, kardus, suara

## Stopword removal

In [15]:
def stopword_removal(lemmatized_):
    clean = []
    for i in lemmatized_:
        if i not in stopwords:
            clean.append(i)

    return clean

In [16]:
clean = lemmatized.apply(stopword_removal)

In [17]:
clean.tail()

14543    [nama, sumpah, janji, harga, rezim, moral, selenggara, pemilu]                                                                                 
14544    [wasit, serah, main, wasit, pimpin, tanding, putus, halo, angkat, tangan, tidak_mampu, wasit, wni, negeri, kompeten, wasit, pemilu, kali]      
14545    [arti, kampanye, lihat, undang, pemilu, nama, kampanye, visi, misi, salah, baca]                                                               
14546    [diskusi, kemarin, bahas, kotak, kardus, suara, tarik, kotak, suara, kardus, daun, pisangpun, trust, masyarakat, selenggara, pemilu, tidak_ada]
14547    [profesor, mantan, komisioner, lupa, nama, ketua, anggap, pemilu, tanam, jagung, kebun]                                                        
Name: clean, dtype: object

In [18]:
df['clean'] = clean

In [19]:
clean.head()

0    [korban, kalah, loloskan, pemilu, rentak]                                                                                                     
1    [fenomena, kuat, lembaga, laksana, baca, munculnya, langgar, salah, orang, tinggal, gara, pemilu, aneka, ragam, heran, muncul, gugat, laksana]
2    [pemilu, ulang, broo, menang, jokowi, sidiq, tabliq, amanah, fatonah]                                                                         
3    [terap, selesai, minggu, korban, romusha, rodi, proyek, pemilu, situng, kpu, system, audit, trial, validasi]                                  
4    [ayo, simak, orasi, ketua, tugas, anti, curang, pemilu]                                                                                       
Name: clean, dtype: object

In [20]:
df.to_csv(os.path.join(os.pardir,os.pardir,"data","05_out","50-KCore-tweet-clean.csv"), index=False)